In [20]:
%%bash
pip install --upgrade pip
pip install smart_open minecart
pip install textract-trp
pip install jupyterthemes

In [38]:
# create a darker background template (better for my eyes)
from jupyterthemes.stylefx import set_nb_theme
# set_nb_theme('chesterish')

In [22]:
import time 
import re
import os
import trp
import random 
import boto3
import minecart
import logging 

import numpy as np
import pandas as pd

from smart_open import open
from sagemaker.session import Session
from io import BytesIO

%matplotlib inline

In [23]:
# initiate s3 bucket and corresponding data folder
bucket = "ran-s3-systemic-risk"
data_folder ="FOCUS-OCR/SubsetTest/"

# Amazon Textract client and Sagemaker session
textract = boto3.client('textract')
s3 = boto3.client('s3')
session = Session()

# discover all of the pdfs that you want to parse
paths = np.array(session.list_s3_files(bucket, data_folder))

In [ ]:
# build log file if one does not exist
logging.basicConfig(filename='error.log')

# AWS Asynchronous Textract Script (requesting Job)
**Content modified from Amazon AWS Textract repository (refer to [URL](https://github.com/aws-samples/amazon-textract-code-samples/blob/master/python/12-pdf-text.py) below)** 

In [24]:
def startJob(s3BucketName:str, objectName:str) -> str:
    """
    Starts a Textract job on AWS server 
    """
    response = None
    client = boto3.client('textract')
    
    # issue response to AWS to start Textract job for table analysis 
    response = client.start_document_analysis(
        DocumentLocation={
            'S3Object': {
                'Bucket': s3BucketName,
                'Name': objectName
            }
        },
        FeatureTypes=['TABLES']
    )
    
    # return response job ID for service
    return response["JobId"]

In [25]:
def isJobComplete(jobId:str) -> str:
    """
    Tracks the completion status of the Textract job when qued
    """
    time.sleep(5)
    client = boto3.client('textract')
    response = client.get_document_analysis(JobId=jobId)
    status = response["JobStatus"]
    print("Job status: {}".format(status))
    
    # check current status of AWS job (ask server every 5 seconds for data)
    while(status == "IN_PROGRESS"):
        time.sleep(5)                   # lag before reporting status
        response = client.get_document_analysis(JobId=jobId)
        status = response["JobStatus"]
        print("Job status: {}".format(status))
    
    return status

In [26]:
def getJobResults(jobId:str) -> list:
    """
    Returns the contents of the Textract job, after completion status met
    """
    pages = []          # initialize list object to track pages

    client = boto3.client('textract')
    response = client.get_document_analysis(JobId=jobId)
    
    pages.append(response)
    print("Resultset page recieved: {}".format(len(pages)))
    
    nextToken = None
    if('NextToken' in response):
        nextToken = response['NextToken']
    
    # iterate through the pages and append to response figure
    while(nextToken):
        response = client.get_document_analysis(JobId=jobId, NextToken=nextToken)
        pages.append(response)
        print("Resultset page recieved: {}".format(len(pages)))
        nextToken = None
        if('NextToken' in response):
            nextToken = response['NextToken']

    return pages

In [27]:
def runJob(bucket:str, key:str):
    """
    Function designed to call an AWS Textract 
    """
    # S3 storage for files on AWS site   
    jobId = startJob(bucket, key)   # intialize Textract job 
    print("Started job with id: {}".format(jobId))

    # if job is complete from AWS return response object 
    if(isJobComplete(jobId)):
        response = getJobResults(jobId)
        
    return response

# OCR Wrapper Functions
**The scripts perform an OCR job from AWS Textract, converting tabular data into dataframes**

In [28]:
def trp2df(table:trp.Table) -> pd.DataFrame:
    """
    Function designed to convert a trp table into a dataframe
    :param table: a trp table object parsed from a pdf  
    :return: a DataFrame object housing a textracted trp table
    
    Complexity -> O(n^2)
    """
    N = len(table.rows)               # number of rows in table
    M = len(table.rows[0].cells)      # number of columns in table
    arr = [0]*N
    
    # itterate through each row within the provided table
    for row in range(N):
        
        # strip the text from the cell references to construct (N X M) matrix
        arr[row] = [table.rows[row].cells[col].text.strip() for col in range(M)]
        
    return pd.DataFrame(arr)

In [29]:
def readPDF(response:list) -> pd.DataFrame:
    """
    Function to transform AWS Textract object to a pdf
    :param response: AWS Textract response object
    """
    # in the event multiple tables detected on one page (concat them)
    catDF = []
    
    # format the Textract response type 
    doc = trp.Document(response)
    
    # iterate through document pages
    for page in doc.pages:
        # itterate through page tables
        for table in page.tables: 
            # convert trp-table into dataframe object
            df = trp2df(table)
            
            # remove columns that are completly empty (column 0 = line items)
            empty_cols = [col for col in df.columns if (df[col] == '').all()]
            df = df.drop(empty_cols, axis=1)
            
            # make sure dataframe is not empty  
            if df.values.size > 0:
                colIndex = df.columns[0]

                # check for the word "cash" in a string at the begining, ignoring case sensitivity
                assetCheck = df[colIndex].str.contains('^Cash', regex=True, flags=re.IGNORECASE)

                # check for the word "Liabilities" in a string at the end, ignoring case sensitivity
                debtCheck1 = df[colIndex].str.contains('Liabilities$|^Liabilities', 
                                                      regex=True, flags=re.IGNORECASE)
                debtCheck2 = df[colIndex].str.contains('Liability$|^Liability', 
                                                      regex=True, flags=re.IGNORECASE)

                # check if the key words have been found 
                check1 = df[assetCheck | debtCheck1 | debtCheck2].empty
                check2 = debtCheck1[debtCheck1 == True].empty
                check3 = debtCheck2[debtCheck2 == True].empty

                # if figure matched exactly, we assume this is balance sheet 
                if not check1:
                    catDF.append(df)

                    if check2 == False or check3 == False:  
                        return pd.concat(catDF)
                

## Extract Balance Sheet information

In [30]:
subsetFolder = np.array(session.list_s3_files(bucket, 'Input/SubSets/'))[1:]

In [31]:
subsetFolder

array(['Input/SubSets/1000146-02_subset.pdf',
       'Input/SubSets/1000146-03_subset.pdf',
       'Input/SubSets/1000146-04_subset.pdf', ...,
       'Input/SubSets/99947-17_subset.pdf',
       'Input/SubSets/99947-18_subset.pdf',
       'Input/SubSets/99947-19_subset.pdf'], dtype='<U35')

In [32]:
# J.P. MORGAN SECURITIES LLC, GOLDMAN SACHS & CO. LLC, MORGAN STANLEY & CO. LLC, CITIGROUP GLOBAL MARKETS INC.
# NOMURA SECURITIES INTERNATIONAL, INC., WELLS FARGO SECURITIES, LLC, BARCLAYS CAPITAL INC.
# HSBC SECURITIES (USA) INC., DEUTSCHE BANK SECURITIES INC.
big_banks = ['782124', '42352', '68136', '91154', '72267', '1224385', '851376', '853784', '58056']

# list of big banks paths, we search for postive value denoting occurence
bankPaths = [path for cik in big_banks for path in subsetFolder if str.find(path, cik) > 0]

In [33]:
np.array(bankPaths)[:10]

array(['Input/SubSets/782124-02_subset.pdf',
       'Input/SubSets/782124-03_subset.pdf',
       'Input/SubSets/782124-04_subset.pdf',
       'Input/SubSets/782124-05_subset.pdf',
       'Input/SubSets/782124-06_subset.pdf',
       'Input/SubSets/782124-07_subset.pdf',
       'Input/SubSets/782124-08_subset.pdf',
       'Input/SubSets/782124-09_subset.pdf',
       'Input/SubSets/782124-10_subset.pdf',
       'Input/SubSets/782124-11_subset.pdf'], dtype='<U35')

In [34]:
print('Total files needed to be converted {}'.format(subsetFolder.size))

Total files needed to be converted 83129


In [35]:
# script to perform OCR (using Textract) for X-17A-5 subsets
outFolder = 'Output/BalanceSheet/'
csvDirectory = np.array(session.list_s3_files(bucket, outFolder))

# iterate through X-17A-5 subsets stored in s3 
for i, key in enumerate(bankPaths): #enumerate(subsetFolder[20500:25000]):
    # baseFile name (CIK)-{Year}
    baseFile = key.split('/')[-1].split('_')[0]
    fileName = baseFile+'.csv'
    print('\nFile {}'.format(i))
    # if file is not found in directory 
    if outFolder+fileName not in csvDirectory:

        # temporary data frame object for balance sheet information
        res = runJob("ran-s3-systemic-risk", key)
        
        if res[0]['JobStatus'] != 'FAILED':
            tempDF = readPDF(res)
            print(tempDF)
            # checks for type of return, if none then we log an error
            if type(tempDF) == pd.DataFrame:
                # writing data frame to .csv file
                tempDF.to_csv(fileName)

                # the folder used to export to
                xFolder = 'Output/BalanceSheet/'+fileName

                # save contents to AWS S3 bucket
                with open(fileName, 'rb') as data:
                    s3.put_object(Bucket=bucket, Key=xFolder, Body=data)

                # remove local file after it has been created
                os.remove(fileName)
                print('\tSaved {} file to s3 bucket'.format(baseFile+'.csv'))
            else:
                logging.info('No Balance Sheet found in {}'.format(baseFile))
        else:
            logging.info('Could not parse {}'.format(baseFile))
    else:
        print('{} has been downloaded'.format(fileName))

print('==========================\nOCR is completed')


File 0
Started job with id: 8807872b8fd0f2293141dab33c098a67925781adcdf069c49432babb9e382b27
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: SUCCEEDED
Resultset page recieved: 1
Resultset page recieved: 2
Resultset page recieved: 3
Resultset page recieved: 4
Resultset page recieved: 5
                                                    0              1
0                                              ASSETS               
1                           Cash and cash equivalents      $ 222,336
2   Cash and securities deposited with clearing or...               
3              in compliance with federal regulations      9,071,138
4     Securities purchased under agreements to resell     34,764,794
5                                 Securities borrowed     51,094,781
6                   Securities received as collateral      3,037,956
7                                        Receivables:               
8                                           Customers   

Started job with id: 3c58fb2938a89394fcc3534f01417e2c352d4aca0d1264ecfd1e3702b72bcc04
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: SUCCEEDED
Resultset page recieved: 1
Resultset page recieved: 2
Resultset page recieved: 3
Resultset page recieved: 4
Resultset page recieved: 5
                                                    0              1
0                                              ASSETS               
1                           Cash and cash equivalents    $ 2,327,083
2   Cash and securities deposited with clearing or...               
3              in compliance with federal regulations      4,170,898
4     Securities purchased under agreements to resell     41,150,845
5                                 Securities borrowed     70,753,053
6                   Securities received as collateral      8,625,128
7                                        Receivables:               
8                                           Customers     26,288

	Saved 782124-07.csv file to s3 bucket

File 6
Started job with id: 04c548c827408c3be765cd7fa393d92d237fccbfa62b10bbf07327ecceca9a71
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: SUCCEEDED
Resultset page recieved: 1
Resultset page recieved: 2
Resultset page recieved: 3
Resultset page recieved: 4
Resultset page recieved: 5
                                                    0                  1
0                                              ASSETS                   
1                           Cash and cash equivalents          $ 781,197
2   Cash and securities deposited with clearing or...                   
3              in compliance with federal regulations         11,593,568
4     Securities purchased under agreements to resell         27,815,064
5                                 Securities borrowed   

	Saved 782124-10.csv file to s3 bucket

File 9
Started job with id: 833de87217083f5d04a033c93e52aa785daa6e01f01edef771418701502ac0d2
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: SUCCEEDED
Resultset page recieved: 1
Resultset page recieved: 2
Resultset page recieved: 3
Resultset page recieved: 4
                                                    0          1
0                                              Assets           
1                                                Cash      $ 555
2   Cash and securities segregated under federal a...      1,583
3        Securities purchased under resale agreements    129,951
4   (included $423 at fair value at December 31, 2...           
5                                 Securities borrowed     78,954
6                   Securities received as collateral      5,379
7   Receivable from brokers, dealers and clearing ...      5,712
8                           Receivable from customers     32,723
9              

Started job with id: 04d02d95dd8c1606c6816f42e5bf1ce4d272f0957e69095477723a8bfe39f52b
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: SUCCEEDED
Resultset page recieved: 1
Resultset page recieved: 2
None
No Balance Sheet found in 782124-14

File 13
Started job with id: bde9e06f3103d5f0b5fc31ee6965a818ca6fa4ba75594038817c4b50bff762aa
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: SUCCEEDED
Resultset page recieved: 1
Resultset page recieved: 2
Resultset page recieved: 3
Resultset page recieved: 4
Resultset page recieved: 5
Resultset page recieved: 6
                                                    0        1
0                                                      Page(s)
1                        Independent Auditor's Report         
2       Consolidated Statement of Financial Condition        2
3 

...

Started job with id: 6cbbe3ca6cde4dfd052966945adfbc37d75d760875c65c215fd192f08be6c982
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: SUCCEEDED
Resultset page recieved: 1
Resultset page recieved: 2
Resultset page recieved: 3
Resultset page recieved: 4
                                                    0              1
0   Cash, cash equivalents, and cash segregated fo...      $ 392,885
1                           Receivable from customers      5,564,333
2                                 Securities borrowed     78,677,547
3   Receivables from brokers, dealers, and clearin...     10,275,917
4     Securities purchased under agreements to resell     49,457,744
5                  Securities owned, at market value:               
6              U.S. Government and agency obligations     24,986,249
7   U.S. Government obligations pledged as collateral     17,029,717
8                State and municipal bond obligations         41,56

Started job with id: 1f60a5e5006911d685d580c8b1f42169748c9a61044a0f637cd064dce689943f
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: SUCCEEDED
Resultset page recieved: 1
Resultset page recieved: 2
Resultset page recieved: 3
Resultset page recieved: 4
                                                    1              2
0                                              Assets               
1                           Cash and cash equivalents    $ 1,597,952
2   Cash and securities segregated for benefit of ...      7,870,687
3                  Securities financing transactions:               
4     Securities purchased under agreements to resell     61,068,432
5                                 Securities borrowed    118,171,330
6                                                        179,239,762
7   Financial instruments owned, at market value (...    103,145,058
8                                        Receivables:               
9                     

	Saved 58056-09.csv file to s3 bucket

File 154
Started job with id: 2e40d9b56fd5f259a075365e67cdf55db54ca43cb8c1899fbe85001ad232d97b
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: SUCCEEDED
Resultset page recieved: 1
Resultset page recieved: 2
Resultset page recieved: 3
Resultset page recieved: 4
Resultset page recieved: 5
                                                    0  \
0                                              Assets   
1                           Cash and cash equivalents   
2   Cash and securities segregated for benefit of ...   
3   Securities financing transactions: Securities ...   
4                                                       
5   Financial instruments owned, at fair value (in...   
6                              Receivables: Customers   
7        Brokers, dealers, and clearing organizations   
8                                               Other   
9                                          

Started job with id: 01d5360de5a61298f2d399d71d88488771ac41b702202bb56b40818b058b28da
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: SUCCEEDED
Resultset page recieved: 1
Resultset page recieved: 2
Resultset page recieved: 3
Resultset page recieved: 4
                                                    0          1
0   Cash and cash equivalents (includes cash equiv...    $ 1,219
1   Cash and securities segregated for benefit of ...     11,725
2           Collateralized agreements and financings:           
3   Securities purchased under agreements to resel...     51,173
4   Securities borrowed (includes $43,452 at fair ...     67,235
5                                                        118,408
6   Financial instruments owned, at fair value (in...     61,094
7                                        Receivables:           
8                                           Customers      1,638
9                                     

Started job with id: f42bf0b3cd51674ab35c603be709a0a8730ddc4f3f4cfa933346fb02ede0ac60
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: SUCCEEDED
Resultset page recieved: 1
Resultset page recieved: 2
Resultset page recieved: 3
Resultset page recieved: 4
Resultset page recieved: 5
Resultset page recieved: 6
None
No Balance Sheet found in 58056-17

File 161
Started job with id: 204da03d82f544a4d95461cbe8937be64e8e2ed143c7c590711e15343382258f
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: SUCCEEDED
Resultset page recieved: 1
Resultset page recieved: 2
Resultset page recieved: 3
Resultset page recieved: 4
Resultset page recieved: 5
                                                    0                1
0   Cash and cash equivalents (includes cash equiv...          $ 1,060
1   Cash and securities segregated for benefit of ...            3,402
2           Collateralized agreements and financings:   